In [1]:
from load_utils import prepare_data
from train_utils import *
from seq2seq_model import *
import numpy as np
import torch
from tqdm.notebook import tnrange, tqdm_notebook
import datetime

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
q, a, pairs, vector = prepare_data('test', 'glove.42B.300d/glove.42B.300d.txt', small=True)

Reading test -------
Read 4041 sentence pairs
Counting words
Counted words:
In questions: 6820 words
In answers: 4756 words


In [4]:
matrix_len = q.n_words
weights_matrix = np.zeros((matrix_len, 300))
word_found = 0
for i, word in enumerate(q.word2index):
    try:
        weights_matrix[i] = vector[word]
    except:
        continue

In [5]:
from matplotlib import pyplot as plt

class LivePlotNotebook(object):
    """
    Live plot using %matplotlib notebook in jupyter notebook
    
    Usage:
    ```
    import time
    liveplot = LivePlotNotebook()
    x=np.random.random((10,))
    for i in range(10):
        time.sleep(1)
        liveplot.update(
            x=x+np.random.random(x.shape)/10,
            actions=np.random.randint(0, 3, size=(10,))
        )
    ```
    
    url:
    """

    def __init__(self):
        %matplotlib notebook
        fig,ax = plt.subplots(1,1)
        
        ax.plot([0]*20, label='price')
        ax.plot([1]*20, [1]*20, 'o', ms=12,c='gray', label='hold')
        
        
    
        ax.legend()
        ax.set_xlabel('timesteps')
        ax.grid()
        ax.set_title('actions')
        
        self.ax = ax
        self.fig = fig

    def update(self, x, actions):             
        # update price
        line = self.ax.lines[0]
        line.set_xdata(range(len(x)))
        line.set_ydata(x)
        
        # update action plots
        for i, line in enumerate(self.ax.lines[1:]):
            line.set_xdata(np.argwhere(actions==i).T)
            line.set_ydata(x[actions==i])
            line.set_marker(['o','^','v'][i])

        # update limits
        self.ax.set_xlim(0, len(actions))
        self.ax.set_ylim(x.min(), x.max())

        self.fig.canvas.draw()
        

In [6]:
criterion = nn.NLLLoss
optimizer = 'Adam'
batch_size = 32

In [7]:
model = Seq2Seq(batch_size, q.n_words, a.n_words, 300, 100, weights_matrix, 0.5, device, criterion)

C:\Users\maren\Anaconda3\envs\deeplearning\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [8]:
for epoch in tnrange(2, desc="Total epochs: "):

    print(f'Epoch {epoch + 1}: {datetime.datetime.now()}')
  
    # Calculte loss
    loss = epoch_train(model, optimizer, batch_size, pairs, q, device)

    print(f'Loss: {loss}')

    # Calculate accuracy
    accuracy = epoch_accuray(model, batch_size, pairs, a, device)

    print(f'Train accuracy: {accuracy}')

    # Calculate BLEU Score
    BLEU_model = CalculateBleu(model, batch_size, pairs, q, a, device)
    bleu_score = BLEU_model.score()

    print(f'BLUE score: {bleu_score}')

    # Try to do interactive plot
    


print(f"Optimization ended successfully")


Total epochs:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1: 2021-09-01 07:16:13.905749


RuntimeError: Length of all samples has to be greater than 0, but found an element in 'lengths' that is <= 0

In [9]:
pairs

[['hey man you wanna buy some weed ?', 'some what ?'],
 ['weed ! you know ? pot ganja mary jane some chronic !',
  'oh umm no thanks .'],
 ['i also have blow if you prefer to do a few lines .', 'no i am ok really .'],
 ['come on man ! i even got dope and acid ! try some !',
  'do you really have all of these drugs ? where do you get them from ?'],
 ['i got my connections ! just tell me what you want and i ll even give you one ounce for free .',
  'sounds good ! let s see i want .'],
 ['yeah ?',
  'i want you to put your hands behind your head ! you are under arrest !'],
 ['', 'what for ?'],
 ['the taxi drivers are on strike again .', 'it is really a hot potato .'],
 ['they want the government to reduce the price of the gasoline .',
  "that's excellent . how have you managed that ?"],
 ['', 'what does that mean exactly ?'],
 ["we've managed to reduce our energy consumption in our factory by about per cent in the last two years .",
  'what other sources of energy do you use ?'],
 ["mainl

In [10]:
pairs = [['PAD', line[1]] if line[0] == '' else line for line in pairs]

In [11]:
pairs

[['hey man you wanna buy some weed ?', 'some what ?'],
 ['weed ! you know ? pot ganja mary jane some chronic !',
  'oh umm no thanks .'],
 ['i also have blow if you prefer to do a few lines .', 'no i am ok really .'],
 ['come on man ! i even got dope and acid ! try some !',
  'do you really have all of these drugs ? where do you get them from ?'],
 ['i got my connections ! just tell me what you want and i ll even give you one ounce for free .',
  'sounds good ! let s see i want .'],
 ['yeah ?',
  'i want you to put your hands behind your head ! you are under arrest !'],
 ['PAD', 'what for ?'],
 ['the taxi drivers are on strike again .', 'it is really a hot potato .'],
 ['they want the government to reduce the price of the gasoline .',
  "that's excellent . how have you managed that ?"],
 ['PAD', 'what does that mean exactly ?'],
 ["we've managed to reduce our energy consumption in our factory by about per cent in the last two years .",
  'what other sources of energy do you use ?'],
 [

In [12]:
pairs = [[line[0], 'PAD'] if line[1] == '' else line for line in pairs]

In [13]:
pairs

[['hey man you wanna buy some weed ?', 'some what ?'],
 ['weed ! you know ? pot ganja mary jane some chronic !',
  'oh umm no thanks .'],
 ['i also have blow if you prefer to do a few lines .', 'no i am ok really .'],
 ['come on man ! i even got dope and acid ! try some !',
  'do you really have all of these drugs ? where do you get them from ?'],
 ['i got my connections ! just tell me what you want and i ll even give you one ounce for free .',
  'sounds good ! let s see i want .'],
 ['yeah ?',
  'i want you to put your hands behind your head ! you are under arrest !'],
 ['PAD', 'what for ?'],
 ['the taxi drivers are on strike again .', 'it is really a hot potato .'],
 ['they want the government to reduce the price of the gasoline .',
  "that's excellent . how have you managed that ?"],
 ['PAD', 'what does that mean exactly ?'],
 ["we've managed to reduce our energy consumption in our factory by about per cent in the last two years .",
  'what other sources of energy do you use ?'],
 [